In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census

# Census & gmaps API Keys
from config import (api_key, gkey)
c = Census(api_key, year=2018)

In [2]:
# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E",
                          "B23025_005E"), {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_005E": "Unemployment Count",
                                      "NAME": "Name", "zip code tabulation area": "zip_code"})

# # Add in Poverty Rate (Poverty Count / Population)
# census_pd["Poverty Rate"] = 100 * \
#     census_pd["Poverty Count"].astype(
#         int) / census_pd["Population"].astype(int)

# # Add in Unemployment Rate (Unemployment Count / Population)
# census_pd["Unemployment Rate"] = 100 * \
#     census_pd["Unemployment Count"].astype(
#         int) / census_pd["Population"].astype(int)

# # Final DataFrame
# census_pd = census_pd[["zip_code", "Name", "Population", "Median Age", "Household Income",
#                        "Per Capita Income", "Poverty Count", "Poverty Rate", "Unemployment Rate"]]

# # Visualize
# print(len(census_pd))

census_pd.head()

,Name,Household Income,Population,Median Age,Per Capita Income,Poverty Count,Unemployment Count,zip_code
0,ZCTA5 00601,13092.0,17242.0,40.5,6999.0,10772.0,2316.0,00601
1,ZCTA5 00602,16358.0,38442.0,42.3,9277.0,19611.0,1927.0,00602
2,ZCTA5 00603,16603.0,48814.0,41.1,11307.0,24337.0,3124.0,00603
3,ZCTA5 00606,12832.0,6437.0,43.3,5943.0,4163.0,230.0,00606
4,ZCTA5 00610,19309.0,27073.0,42.1,10220.0,11724.0,1290.0,00610


In [3]:
# Save as a csv
# Note to avoid any issues later, use encoding="utf-8"
census_pd.to_csv("census_data.csv", encoding="utf-8", index=False)

In [ ]:
# Read in the csv containing state centroid coordinates
centroids = pd.read_csv("../Resources/state_centroids.csv")
centroids.head()

In [ ]:
# Merge the datasets using the sate columns
census_data = pd.merge(census_pd, centroids, how="left", left_on="Name", right_on="State")

# Save the updated dataframe as a csv
census_data.to_csv("../Resources/state_census_data.csv", encoding="utf-8", index=False)
census_data.head()

In [ ]:
# Convert poverty rate as a list
# Convert bank rate to list
poverty_rate = census_pd["Poverty Rate"].tolist()

In [ ]:
marker_locations = census_data[["Latitude", "Longitude"]]

fig = gmaps.figure()

markers = gmaps.marker_layer(marker_locations,
    info_box_content=[f"Poverty Rate: {rate}" for rate in poverty_rate])
fig.add_layer(markers)
fig